In [ ]:
%matplotlib inline
from SALib.sample import saltelli
from SALib.analyze import sobol as sobol_analyze
from mesa.batchrunner import batch_run
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import clear_output
import pickle
from joblib import Parallel, delayed
from SALib.sample import sobol
from IPython.display import clear_output
import numpy as np
from itertools import product
import time

from model import SugarscapeG1mt

### Sensitivity Analysis - Problem

In [ ]:
problem = {
    "num_vars": 15,
    "names": [
        "initial_population",
        "endowment_min",
        "endowment_max",
        "vision_min",
        "vision_max",
        "metabolism_min",
        "metabolism_max",
        "wealth_tax_system",
        "flat_rate",
        "wealth_tax_period",
        "income_tax_system",
        "income_tax_flat_rate",
        "p_copy",
        "p_mutate",
        "enable_staghunt"
    ],
    "bounds": [
        [50,   500],    # initial_population
        [0,    10],    # endowment_min
        [15,   30],    # endowment_max
        [1,     10],    # vision_min
        [1,     10],    # vision_max
        [1,     10],    # metabolism_min
        [2,     6],    # metabolism_max
        [0,     3],     # wealth_tax_system
        [0.0,   0.10],  # flat_rate
        [1,     50],    # wealth_tax_period
        [0,     3],     # income_tax_system
        [0.0,   0.20],  # income_tax_flat_rate
        [0.0,   1.0],   # p_copy
        [0.0,   0.20],  # p_mutate
        [0,     1]      # enable_staghunt
    ]
}

model_reporters = {
    "Gini": lambda m:m.datacollector.get_model_vars_dataframe()['Gini'].iloc[-1],
    "Avg Wealth": lambda m:m.datacollector.get_model_vars_dataframe()['Average Wealth'].iloc[-1],

}

wealth_tax_map = {
    0: "none",  # No wealth tax
    1: "proportional",  # Proportional
    2: "progressive",  # Progressive tax
    3: "degressive"   # Regressive tax
}

income_tax_map = {
    0: "none",  # No income tax
    1: "proportional",  # Proportional income tax
    2: "progressive",  # Progressive income tax
    3: "degressive"   # Regressive income tax
}

integer_vars = {
    "initial_population",
    "endowment_min",
    "endowment_max",
    "vision_min",
    "vision_max",
    "metabolism_min",
    "metabolism_max",
    "wealth_tax_period",
    "wealth_tax_system",
    "income_tax_system",
    "enable_staghunt"
}

discrete_factors = {
    "wealth_tax_system": [0, 1, 2, 3],
    "income_tax_system": [0, 1, 2, 3],
    "enable_staghunt":   [0, 1],
}
# Build reduced Sobol problem for the other 12 vars
cont_vars   = [n for n in problem["names"] if n not in discrete_factors]
cont_bounds = [
    b for (n, b) in zip(problem["names"], problem["bounds"])
         if n not in discrete_factors
]
reduced_problem = {
    "num_vars": len(cont_vars),
    "names":    cont_vars,
    "bounds":   cont_bounds
}

In [ ]:

# --- 2) OFAT (one-factor-at-a-time) sweep ---
replicates   = 2
max_steps    = 10
distinct_OAT = 2

test_vars = ["initial_population", "flat_rate"]

OAT_results = {}
# for i, var in enumerate(problem["names"]):
for var in test_vars:
    i = problem["names"].index(var)
    low, high = problem["bounds"][i]

    # 5a) Categorical parameters: use the map values
    if var == "wealth_tax_regime":
        samples = list(wealth_tax_map.values())
    elif var == "income_tax_system":
        samples = list(income_tax_map.values())

    # 5b) Integer parameters: linspace with dtype=int
    elif var in integer_vars:
        samples = np.linspace(low, high, distinct_OAT, dtype=int)

    # 5c) All other continuous parameters
    else:
        samples = np.linspace(low, high, distinct_OAT)

    # 6) Run the batch
    results = batch_run(
        SugarscapeG1mt,
        parameters={var: samples},
        iterations=replicates,
        max_steps=max_steps,
        data_collection_period=1,
        display_progress=True
    )

    # 7) Collect into a DataFrame
    df = pd.DataFrame(results)
    # print(df.columns.tolist())    # ← inspect the real column names
    # print(df.head())       
    # keep only the columns we care about:
    #   - the varied var
    #   - the run number
    #   - the final reporter ("Gini")
    OAT_results[var] = df[[var, "RunId", "Gini"]]
for var, df in OAT_results.items():
    print(f"\n=== {var} ===")
    print(df)

In [ ]:
for var in ["initial_population", "flat_rate"]:
    df = OAT_results[var]   # this DF has columns [var, "Gini"]

    # Group by the parameter value
    grp = df.groupby(var)["Gini"]
    x    = grp.mean().index.astype(float)
    y    = grp.mean().values
    sem  = grp.sem().values
    ci   = 1.96 * sem    # 95% interval

    plt.figure(figsize=(6,4))
    plt.plot(x, y, marker="o")
    plt.fill_between(x, y - ci, y + ci, alpha=0.3)
    plt.xlabel(var)
    plt.ylabel("Final Gini")
    plt.title(f"OAT: {var} → Gini\n(n={len(df)} runs)")
    plt.tight_layout()
    plt.show()

### Global Sensitivity Analysis - Sobol (Parallel)

In [ ]:
measure = "Gini" # "Gini" # or "Avg Wealth"
second_order = True  # or: "True"Oh

# --- 3) Sobol GSA ---
replicates = 2
max_steps = 100
distinct_SA = 2

In [ ]:

param_values = sobol.sample(problem, distinct_SA, calc_second_order=second_order)
total_runs = replicates * len(param_values)
print(f"Total runs for Sobol GSA: {total_runs}")

# --- Helper function to run a single model execution ---
def run_single_sim(run_id, vals):
    vals = list(vals)
    params = {}
    for name, val in zip(problem["names"], vals):
        if name in integer_vars:
            params[name] = int(round(val))
        elif name == "wealth_tax_system":
            params[name] = wealth_tax_map[int(round(val))]
        elif name == "income_tax_system":
            params[name] = income_tax_map[int(round(val))]
        else:
            params[name] = float(val)

    for low_key, high_key in [
        ("endowment_min", "endowment_max"),
        ("vision_min", "vision_max"),
        ("metabolism_min", "metabolism_max"),
    ]:
        lo = params[low_key]
        hi = params[high_key]
        lo, hi = int(min(lo, hi)), int(max(lo, hi))
        params[low_key], params[high_key] = lo, hi

    params["enable_staghunt"] = bool(params["enable_staghunt"])
    out = batch_run(
        SugarscapeG1mt,
        parameters=params,
        iterations=1,
        max_steps=max_steps,
        data_collection_period=-1,
        display_progress=False
    )[0]

    return {
        **params,
        "RunId": run_id,
        "Gini": out["Gini"],
        "Avg Wealth": out["Average Wealth"] }

# --- Create tasks ---
tasks = [(run_id, vals) for run_id in range(replicates) for vals in param_values]

# --- Run in parallel ---
results = Parallel(n_jobs=-1, verbose=10)(delayed(run_single_sim)(run_id, vals) for run_id, vals in tasks)

# --- Done ---
records = results
print(f"Completed all {total_runs} runs.")

# --- 5) Save results ---
output_path = "sobol_gsa_results.pkl"
with open(output_path, "wb") as f:
    pickle.dump(results, f)

print(f"\n All {total_runs} runs completed and saved to '{output_path}'.")


In [ ]:
# 7) Build the DataFrame and extract Y
with open("sobol_gsa_results.pkl", "rb") as f:
    records = pickle.load(f)
df = pd.DataFrame(records)
Y  = df[str(measure)].values

# 8) Perform Sobol analysis
Si = sobol.analyze(
    problem,
    Y,
    calc_second_order=second_order,
    print_to_console=False
)

fig, axes = plt.subplots(1, 2, figsize=(10,4), tight_layout=True)

# First‐order
axes[0].bar(problem["names"], Si["S1"])
axes[0].set_title(f"First‐order Sobol (S1) \n y = {measure}")
axes[0].set_ylabel("Index")
# rotate labels
axes[0].tick_params(axis="x", labelrotation=45)
# right‐align tick labels
for lbl in axes[0].get_xticklabels():
    lbl.set_ha("right")

# Total‐order
axes[1].bar(problem["names"], Si["ST"])
axes[1].set_title(f"Total‐order Sobol (ST) \n y = {measure}")
axes[1].tick_params(axis="x", labelrotation=45)
for lbl in axes[1].get_xticklabels():
    lbl.set_ha("right")

plt.show()

### Global SA with Discrete settings for tax systems and stag hunt

In [ ]:
def run_mixed(run_id, params):
    # cast ints/floats just as before
    for name in integer_vars:
        if name in params:
            params[name] = int(round(params[name]))
    params["flat_rate"]           = float(params["flat_rate"])
    params["income_tax_flat_rate"]= float(params["income_tax_flat_rate"])
    params["p_copy"]              = float(params["p_copy"])
    params["p_mutate"]            = float(params["p_mutate"])
    # ensure min≤max
    for low, high in [("endowment_min","endowment_max"),
                        ("vision_min",   "vision_max"),
                        ("metabolism_min","metabolism_max")]:
        lo, hi = sorted((params[low], params[high]))
        params[low], params[high] = int(lo), int(hi)
    # map tax‐system integers to names
    params["wealth_tax_system"] = wealth_tax_map[params["wealth_tax_system"]]
    params["income_tax_system"]= income_tax_map[params["income_tax_system"]]
    # run your model
    out = batch_run(
        SugarscapeG1mt,
        parameters=params,
        iterations=1,
        max_steps=max_steps,
        data_collection_period=-1,
        display_progress=False
    )[0]
    return {
        **params,
        "RunId":    run_id,
        "Gini":     out["Gini"],
        "Avg Wealth": out["Average Wealth"]
    }

In [ ]:
import multiprocessing
t0 = time.perf_counter()
_ = run_mixed(*tasks[0])
t = time.perf_counter() - t0

total_runs = replicates * distinct_SA * (len(cont_vars) + 2) * len(discrete_factors["wealth_tax_system"]) \
             * len(discrete_factors["income_tax_system"]) * len(discrete_factors["enable_staghunt"])

C = multiprocessing.cpu_count()

print(f"Estimated wall‐time: {(total_runs * t / C)/60:.1f} minutes on {C} cores")

In [ ]:
all_records = []
for w_sys, i_sys, hunt_flag in product(
        discrete_factors["wealth_tax_system"],
        discrete_factors["income_tax_system"],
        discrete_factors["enable_staghunt"]):

    # 2a) sample continuous part
    cont_samples = sobol.sample(
        reduced_problem,
        N=distinct_SA,
        calc_second_order=second_order
    )

    # 2b) build tasks
    tasks = []
    for run_id, cont_vals in enumerate(cont_samples):
        # merge into a single params dict
        full_params = {n: v for n, v in zip(cont_vars, cont_vals)}
        full_params.update({
            "wealth_tax_system": w_sys,
            "income_tax_system": i_sys,
            "enable_staghunt":   bool(hunt_flag)
        })
        tasks.append((run_id, full_params))
    slice_results = Parallel(n_jobs=-1, verbose=10)(
        delayed(run_mixed)(rid, p) for rid, p in tasks
    )

    # collect
    all_records.append({
    "discrete_setting": (w_sys, i_sys, hunt_flag),
    "samples":          cont_samples,
    "results":          slice_results
    })

print(f"Did {len(all_records)} slices run (should be 4×4×2 = 32)?")
# Save all results
output_path = "discrete_sobol_gsa_results.pkl"

with open(output_path, "wb") as f:
    pickle.dump(all_records, f)
print(f"\nAll {len(all_records)} runs completed and saved to '{output_path}'.")

In [ ]:
summary = []
for entry in all_records:
    w_sys, i_sys, hunt = entry["discrete_setting"]
    ginis = [r["Gini"] for r in entry["results"]]
    aws   = [r["Avg Wealth"] for r in entry["results"]]
    summary.append({
        "wealth_tax":        w_sys,
        "income_tax":        i_sys,
        "staghunt_enabled":  hunt,
        "n_runs":            len(entry["results"]),
        "mean_Gini":         np.mean(ginis),
        "mean_Avg_Wealth":   np.mean(aws)
    })

df = pd.DataFrame(summary)
print(df.head(10)) 

In [ ]:
print(df.shape[0]) 


In [ ]:
with open("discrete_sobol_gsa_results.pkl", "rb") as f:
    all_records = pickle.load(f)
sobol_summary = []
for slice_data in all_records:
    Ys = [r[measure] for r in slice_data["results"]]
    Si = sobol_analyze.analyze(
        reduced_problem, 
        np.array(Ys),
        calc_second_order=second_order,
        print_to_console=False
    )
    sobol_summary.append({
        "setting": slice_data["discrete_setting"],
        "S1":      Si["S1"], 
        "ST":      Si["ST"]
    })


In [ ]:
# 1) Extract parameter names and build arrays
param_names = reduced_problem["names"]
S1_array = np.vstack([entry["S1"] for entry in sobol_summary])   # shape = (n_slices, n_params)
ST_array = np.vstack([entry["ST"] for entry in sobol_summary])   # same

# 2) Compute average indices across all discrete‐factor slices
mean_S1 = S1_array.mean(axis=0)
mean_ST = ST_array.mean(axis=0)

# 3) Pack into a DataFrame (optional, for reference)
df = pd.DataFrame({
    "Parameter": param_names,
    "Mean S1":   mean_S1,
    "Mean ST":   mean_ST
})

# 4a) Plot mean first‐order indices
plt.figure()
plt.bar(df["Parameter"], df["Mean S1"])
plt.xticks(rotation=45, ha="right")
plt.xlabel("Parameter")
plt.ylabel("Mean First‐Order Sobol Index (S1)")
plt.title("Average S1 Across All Discrete Settings")
plt.tight_layout()
plt.show()

# 4b) Plot mean total‐order indices
plt.figure()
plt.bar(df["Parameter"], df["Mean ST"])
plt.xticks(rotation=45, ha="right")
plt.xlabel("Parameter")
plt.ylabel("Mean Total‐Order Sobol Index (ST)")
plt.title("Average ST Across All Discrete Settings")
plt.tight_layout()
plt.show()